import numpy as np import matplotlib.pyplot as plt

# Parameters

nx, ny = 41, 41 \# Number of grid points lx, ly = 1.0, 1.0 \# Length of
the domain in x and y directions dx, dy = lx/(nx-1), ly/(ny-1) \# Grid
spacing nt = 500 \# Number of time steps nu = 0.1 \# Kinematic viscosity
dt = 0.001 \# Time step size rho = 1.0 \# Density (now defined)

# Initialize variables

u = np.zeros((ny, nx)) v = np.zeros((ny, nx)) p = np.zeros((ny, nx)) b =
np.zeros((ny, nx))

# Boundary conditions

def apply_boundary_conditions(u, v): u\[0, :\] = 0 u\[-1, :\] = 0 u\[:,
0\] = 0 u\[:, -1\] = 1 \# Top lid with constant velocity v\[0, :\] = 0
v\[-1, :\] = 0 v\[:, 0\] = 0 v\[:, -1\] = 0

# Build up the RHS of the Poisson equation

def build_up_b(b, u, v, dx, dy, dt): b\[1:-1, 1:-1\] = (rho \* (1/dt *
((u\[1:-1, 2:\] - u\[1:-1, 0:-2\]) / (2 * dx) + (v\[2:, 1:-1\] -
v\[0:-2, 1:-1\]) / (2 \* dy)) - ((u\[1:-1, 2:\] - u\[1:-1, 0:-2\]) / (2
\* dx))**2 - 2 \* ((u\[2:, 1:-1\] - u\[0:-2, 1:-1\]) / (2 \* dy) *
(v\[1:-1, 2:\] - v\[1:-1, 0:-2\]) / (2 * dx)) - ((v\[2:, 1:-1\] -
v\[0:-2, 1:-1\]) / (2 \* dy))**2))

    return b

# Solve the Poisson equation for pressure

def pressure_poisson(p, b, dx, dy): pn = np.empty_like(p) for \_ in
range(50): pn = p.copy() p\[1:-1, 1:-1\] = (((pn\[1:-1, 2:\] + pn\[1:-1,
0:-2\]) \* dy**2 + (pn\[2:, 1:-1\] + pn\[0:-2, 1:-1\]) \* dx**2) / (2 \*
(dx**2 + dy**2)) - dx**2 \* dy**2 / (2 \* (dx**2 + dy**2)) \* b\[1:-1,
1:-1\])

        # Boundary conditions
        p[:, -1] = p[:, -2]
        p[0, :] = p[1, :]
        p[:, 0] = p[:, 1]
        p[-1, :] = 0  # pressure at y = 0

    return p

# Time-stepping loop

for \_ in range(nt): un = u.copy() vn = v.copy()

    b = build_up_b(b, u, v, dx, dy, dt)
    p = pressure_poisson(p, b, dx, dy)

    u[1:-1, 1:-1] = (un[1:-1, 1:-1] -
                     un[1:-1, 1:-1] * dt / dx *
                     (un[1:-1, 1:-1] - un[1:-1, 0:-2]) -
                     vn[1:-1, 1:-1] * dt / dy *
                     (un[1:-1, 1:-1] - un[0:-2, 1:-1]) -
                     dt / (2 * rho * dx) *
                     (p[1:-1, 2:] - p[1:-1, 0:-2]) +
                     nu * (dt / dx**2 *
                           (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2]) +
                           dt / dy**2 *
                           (un[2:, 1:-1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1])))

    v[1:-1, 1:-1] = (vn[1:-1, 1:-1] -
                     un[1:-1, 1:-1] * dt / dx *
                     (vn[1:-1, 1:-1] - vn[1:-1, 0:-2]) -
                     vn[1:-1, 1:-1] * dt / dy *
                     (vn[1:-1, 1:-1] - vn[0:-2, 1:-1]) -
                     dt / (2 * rho * dy) *
                     (p[2:, 1:-1] - p[0:-2, 1:-1]) +
                     nu * (dt / dx**2 *
                           (vn[1:-1, 2:] - 2 * vn[1:-1, 1:-1] + vn[1:-1, 0:-2]) +
                           dt / dy**2 *
                           (vn[2:, 1:-1] - 2 * vn[1:-1, 1:-1] + vn[0:-2, 1:-1])))

    apply_boundary_conditions(u, v)

# Plotting the results

X, Y = np.meshgrid(np.linspace(0, lx, nx), np.linspace(0, ly, ny))

plt.figure(figsize=(11, 7)) plt.quiver(X, Y, u, v) plt.streamplot(X, Y,
u, v, color=np.sqrt(u**2 + v**2), cmap=‘viridis’) plt.xlabel(‘X’)
plt.ylabel(‘Y’) plt.title(‘2D Viscous Flow in a Rectangular Cavity’)
plt.colorbar(label=‘Velocity magnitude’) plt.show()